In [1]:
# import os
# import sys
# if not '/home/anwar/generative-ie' in sys.path:
#     sys.path.insert(0, os.path.abspath('..'))
#     sys.path.insert(0, os.path.abspath('../src'))


In [4]:
!ls workdir/results/

paper_nouns_st
paper_preds_st
paper_roles_st
paper_verbs_st
test-paper_nouns_nltkpos_nounfilter
test-paper_nouns_st_lemminflect
test-paper_nouns_st_lemminflect_stopwords
test-paper_nouns_st_lemminflect_stopwords_nounfilter
test-paper_nouns_st_lemminflect_stopwords_nounfilter_merged
test-paper_nouns_st_lemminflect_stopwords_nounfilter_merged_k50
test-paper_nouns_st_merged
test-paper_nouns_st_pattern
test-paper_nouns_st_pattern_stopwords
test-paper_nouns_st_pattern_stopwords_nounfilter
test-paper_preds_nltkpos_verbfilter
test-paper_preds_nltkpos_vocabfilter
test-paper_preds_nltk_verbfilter
test-paper_preds_nltk_vocabfilter
test-paper_preds_st_lemminflect
test-paper_preds_st_lemminflect_test
test-paper_preds_st_lemminflect_verbfilter
test-paper_preds_st_lemminflect_verbfilter_merged
test-paper_preds_st_lemminflect_verbfilter_merged_k25
test-paper_preds_st_lemminflect_verbfilter_merged_k50
test-paper_preds_st_lemminflect_vocabfilter
test-paper_preds_st_merged
test-paper_preds_st_merged_k50

In [1]:
import os
import pickle
import itertools
import pandas as pd


def load_tps_aps_gps(load_dir_path):
    with open(os.path.join(load_dir_path, 'tps.pkl'), 'rb') as f:
        tps = pickle.load(f)

    with open(os.path.join(load_dir_path, 'aps.pkl'), 'rb') as f:
        aps = pickle.load(f)

    with open(os.path.join(load_dir_path, 'gps.pkl'), 'rb') as f:
        gps = pickle.load(f)
        
    return tps, aps, gps


In [231]:
!ls /raid/anwar/data/fn1.7

ex_dup.pkl		 frame_info.json  ft_frames.csv      lus-wo-ch.csv
ex_frames.csv		 frame_map.json   ft.pkl
ex.pkl			 ft_dup.pkl	  lu_map.json
frame_descriptions.json  ftex_frames.csv  lus-with-ch-r.csv


In [5]:
import json
with open('/raid/anwar/data/fn1.7/frame_descriptions.json', 'r') as f:
    frame_desc = json.load(f)

frame_info = {}
for fns in frame_desc:
    frame_info[fns['fname']] = {'definition': fns['definition']}
#     break

In [6]:
import json

with open('/raid/anwar/data/fn1.7/frame_info.json', 'r') as f:
    frame_desc2 = json.load(f)
frame_desc2


frameroles_info = {}
for fns in frame_desc2:
    frameroles_info[fns['fname']] = {'definition': fns['definition'],
                                'FEs':{fe['name']:{'definition':fe['defn']} for fe in fns['FEs']}
                                }
#     break

In [244]:
frameroles_info['Time_vector']

{'definition': 'An Event is identified as occurring at a particular Distance and in a particular Direction from a Landmark_event.   The lexical units incorporate the Direction and, in some cases, the Landmark_event.  This frame builds on the Direction frame, further specifying the Domain as Time. This frame will receive some of the lexical units currently found in Relative_time.',
 'FEs': {'Landmark_event': {'definition': 'This Frame Element marks expressions that indicate the reference point in which the Event  is anchored.'},
  'Distance': {'definition': 'This FE marks expressions that indicate the amount of time that lies between the Target and the Landmark_event.'},
  'Direction': {'definition': 'This FE marks expressions that indicate whether the Target is prior or subsequent to the Landmark_event on the time line.'},
  'Event': {'definition': 'This FE indicates the Event temporally located with respect to the Landmark_event.'}}}

In [8]:
data_dir = '~/generative-ie/workdir/manual_evaluation/data'
res_dir = '~/generative-ie/workdir/manual_evaluation/result'


In [15]:
P=10

gold_cluster_col='gold_cluster'
luName = 'luName'
luText = 'luText'
luIndex = 'luIndex'


# Verbs
gold_df = pd.read_pickle(f'{data_dir}/verbs_finalpreds_k10.pkl')
load_dir_path = 'workdir/results/test-paper_preds_st_pattern_vocabfilter/gie_swv_test_semiPURExlnet_embs_swvhypers'

# Nouns
# gold_df = pd.read_pickle(f'{data_dir}/nouns_finalpreds_k10.pkl')
# load_dir_path = 'workdir/results/test-paper_nouns_st_pattern_stopwords_nounfilter/gie_swn_test_semiPURExlnet_embs_swnhypers'

#Roles

# gold_cluster_col='gold_cluster_patternlemmatized'
# luName = 'feName'
# luIndex = 'feIndex'
# luText = 'feText'

# gold_df = pd.read_pickle(f'{data_dir}/roles_finalpreds_k10.pkl')
# load_dir_path = 'workdir/results/test-paper_roles_st_pattern/gie_swr_test_semiPURExlnet_embs_swrhypers'

# gold_df = pd.read_pickle(f'{data_dir}/roles_finalpreds_k10_stopwords4.pkl')
# load_dir_path = 'workdir/results/test-paper_roles_st_pattern_stopwords4/gie_swr_test_semiPURExlnet_embs_swrhypers'


tps, aps, gps = load_tps_aps_gps(load_dir_path)

gold_df['tps'] = tps
gold_df['aps'] = aps
gold_df['gps'] = gps
gold_df['tps'] = gold_df['tps'].apply(lambda x: x[:P])
gold_df['aps'] = gold_df['aps'].apply(lambda x: x[:P])
len(tps), len(aps), len(gps)


(75584, 75584, 75584)

In [326]:
gold_df

,frameName,feName,feText,feIndex,sentence,gold_cluster_patternlemmatized,glove,dtwiki,baladd,blc,xlnet_embs,tps,aps,gps
0,Processing_materials,Material,Uranium,"[(25, 31)]",Work Advances at Iranian Uranium Enrichment Site,"[uranium, it, their, plutonium, nuclear, which...","(plutonium, thorium, radioactive, nuclear, fis...","(plutonium, zinc, magnesium, nickel, copper, c...","(plutonium, heu, hexafluoride, thorium, phytoe...","(nuclear, waste, energy, national, petroleum, ...","(oil, coal, gold, fuel, nuclear, gas, diamond,...","(False, False, False, True, True, False, False...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)",22
1,Locale_by_use,Container_possessor,Iranian,"[(17, 23)]",Work Advances at Iranian Uranium Enrichment Site,"[iranian, it, semipalatinsk, national, pavloda...","(iran, lebanese, israeli, arab, kurdish, turki...","(turkish, chinese, russian, british, german, l...","(iraqi, libyan, omani, eritrean, syrian, angol...","(iran, persian, international, islamic, indust...","(iran, tehran, russian, iraqi, ukrainian, kaza...","(False, True, True, False, False, False, False...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)",32
2,Locale_by_use,Locale,Site,"[(44, 47)]",Work Advances at Iranian Uranium Enrichment Site,"[site, facility, center, college, reactor, pla...","(website, web, webpage, web-site, homepage, li...","(location, area, website, ground, complex, bui...","(website, web-site, webiste, sub-site, facilit...","(project, program, facility, source, database,...","(facility, plant, complex, project, area, fiel...","(True, True, True, False, True, True, True, Tr...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)",61
3,Progression,Entity,Work,"[(0, 3)]",Work Advances at Iranian Uranium Enrichment Site,"[work, iran, industry, world, we, it, he, town...","(do, well, job, need, much, way, really, time,...","(employ, study, serve, specialize, spend, live...","(life-work, research, spadework, reserach, gam...","(research, he, she, it, study, project, some, ...","(technology, production, progress, research, s...","(False, False, False, False, False, False, Fal...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)",16
4,Activity_start,Agent,Iran,"[(0, 3)]",Iran has begun preliminary work to install tho...,"[iran, it, taiwan, analyst, i, we, they, wynn,...","(iranian, israel, syria, iraq, tehran, afghani...","(combination, van, wsv, goodnes, program, dham...","(syria, iraq, korea, libya, russia, tehran, pa...","(i, that, it, her, he, have, you, him, she, wh...","(tehran, iranian, china, iraq, russia, libya, ...","(False, False, True, False, False, True, True,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)",53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183247,Roadways,Roadway,boulevard,"[(12, 20)]","In the main boulevard of Havana , they have ar...","[street, path, bridge, boulevard, trail, route...","(avenue, blvd, thoroughfare, promenade, blvd.,...","(roadway, street, avenue, thoroughfare, highwa...","(avenue, promenade, paseo, blvd., rambla, croi...","(square, street, plaza, city, part, port, area...","(street, plaza, avenue, thoroughfare, square, ...","(True, False, True, True, False, False, True, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)",39
183248,Roadways,Roadway,boulevard,"[(56, 64)]",He left the hospital and began to walk slowly ...,"[street, path, bridge, boulevard, trail, route...","(avenue, blvd, thoroughfare, promenade, blvd.,...","(roadway, street, avenue, thoroughfare, highwa...","(avenue, promenade, riverwalk, corniche, crois...","(road, street, avenue, lane, path, corridor, t...","(street, road, avenue, sidewalk, highway, thor...","(True, True, True, True, True, True, True, Tru...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)",39
183249,Roadways,Roadway,boulevards,"[(16, 25)]","Not for him the boulevards of Hollywood , nor ...","[street, path, bridge, boulevard, trail, route...","(thoroughfare, tree-line, promenade, plaza, pa...","(roadway, street, avenue, thoroughfare, highwa...","(sidewalk, promenade, piazza, plaza, street, h...","(beauty, look, charm, smile, power,

In [16]:
import random
random.seed(11)
sample = random.sample([i for i in range(0,len(gold_df))], 50)
# sample

df = gold_df.iloc[sample]
df

,frameName,luName,luText,luIndex,sentence,gold_cluster,fasttext,dt59g,baladd,blc,xlnet_embs,tps,aps,gps
59294,Leadership,preside,presiding,"[(148, 156)]",In an attempt to continue funding its advanced...,"[govern, spearhead, preside, head, administer,...","[superintend, judge, appoint, chairman, office...","[officiate, dissent, live, founder, surrogate,...","[officiate, deliberate, adjudicate, mull, pore...","[reign, rule, hold, lead, helm, govern, head, ...","[reign, oversee, supervise, rule, govern, prev...","(True, False, False, True, True, False, False,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)",10
73370,Placing,drape,draped,"[(65, 70)]",Schools and most offices closed ; taxis flew b...,"[insert, shower, inject, cram, billet, implant...","[swathe, adorn, bedeck, festoon, draping, unfu...","[breathe, reason, bewitch, scream, originate, ...","[festoon, swathe, hang, bedeck, plaster, furl,...","[hang, wave, fly, stream, sling, rip, unfold, ...","[hang, wrapped, adorn, spray, paint, display, ...","(True, False, False, False, False, False, Fals...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)",65
61033,Perception_active,watch,watching,"[(18, 25)]",The companies are watching carefully lest othe...,"[gape, gawk, goggle, peek, peep, feel, peer, s...","[see, enjoy, observe, view, listen, witness, c...","[bike, hike, camp, boat, canoe, kayak, fishes,...","[see, listen, observe, eye, scrutinize, witnes...","[monitor, observe, look, study, guard, listen,...","[observe, listen, view, monitor, wait, see, re...","(True, True, True, False, False, False, False,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)",25
59222,Conduct,misbehave,misbehave,"[(9, 17)]",Children misbehave when they are discouraged a...,"[act, conduct, misbehave, behave]","[behave, scold, punish, spank, mistreat, chast...","[spank, disobey, try, violate, shoplift, find,...","[behave, over-react, overeat, mistreat, undera...","[cry, learn, succeed, play, act, suffer, work,...","[suffer, fail, learn, behave, succeed, disobey...","(False, False, False, True, False, False, Fals...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)",3
66563,Filling,inject,injected,"[(23, 30)]",Some of the males were injected with morphine ...,"[butter, asphalt, surface, anoint, tile, adorn...","[infuse, dose, implant, inoculate, insert, syr...","[add, ingest, inhale, purify, contaminate, pow...","[dose, pump, resuspend, infuse, implant, trans...","[insert, administer, treat, provide, pump, sta...","[insert, pump, treat, spray, administer, infec...","(False, True, False, True, False, False, False...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)",58
24890,Attaching,solder,soldered,"[(26, 33)]",Where the mains leads are soldered to the outp...,"[moor, cinch, agglutinate, solder, shackle, we...","[unsolder, epoxy, braze, weld, socket, spot-we...","[re-form, gild, interest, antique, punch, prec...","[weld, braze, fasten, re-attach, tension, rive...","[sell, deliver, connect, ship, wire, send, use...","[sell, fit, rout, connect, wire, supply, mount...","(False, False, False, True, True, False, True,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)",50
24203,Execute_plan,implement,implemented,"[(111, 121)]",He later produced the Blackburn Report on the ...,"[institute, implement]","[develop, enact, introduce, institute, devise,...","[advertise, license, learn, authorise, prefer,...","[enact, introduce, enforce, deploy, adopt, und...","[enact, adopt, enforce, introduce, institute, ...","[institute, enact, enforce, adopt, execute, pr...","(True, False, False, False, False, False, Fals...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)",1
67096,Coming_to_believe,guess,guess,"[(66, 70)]",I felt angry and humiliated at the time of the...,"[speculate, surmise, deduce, determine, realiz...","[suppose, think, know, figure, presume, say, w...","[think, believe, say, know, realize, realise, ...","[surmise, suppose, presume, reckon, imagine, t...","[imagine, estimate, judge, wonder, tell, pictu...","[suppose, imagine, think, speculate, assume, c...","(False, False, False, True, False, False

In [18]:
dfv = df.copy()

## to match inflection
mainly for lexical units

In [67]:
from lemminflect import getInflection, getAllInflections, getAllLemmas
UTAGS={
'JJ':'ADJ',   
'JJR':'ADJ',  
'JJS':'ADJ',

'RB':'ADV',
'RBR':'ADV',
'RBS':'ADV',

'NN':'NOUN',
'NNS':'NOUN',
'NNP':'PROPN',
'NNPS':'PROPN',

# upos = 'VERB', 'AUX'
'VB':'VERB',
'VBD':'VERB',
'VBG':'VERB',
'VBN':'VERB',
'VBP':'VERB',
'VBZ':'VERB',
'MD':'VERB'
}
def _match_inflection(pred, postag):
    if postag in UTAGS.keys():
        inflected_form = getInflection(pred, tag=postag)
        if inflected_form:
            return inflected_form[0]
    return pred

def match_inflection(predictions, postags):
    return [[_match_inflection(pred, postags[i]) for pred in predictions[i]] for i in tqdm(range(len(predictions)))]



### FOR VERBS

In [ ]:
new_df = pd.DataFrame(columns=['frameName','frame_description', 'gold_terms','original_term','sentence', 'match gold'])
index_col = 'luIndex'
text_col = 'luText'
name_col = 'luName'


for gd, fn, luName, luText, luIndex, sentence, predictions, match_gold in zip(df[gold_cluster_col], df['frameName'], df[name_col], df[text_col], df[index_col], df['sentence'], df['xlnet_embs'], df['tps']):
#     print(len(predictions), len(match_gold))
    inflections = getAllInflections(luName)
    postag = None
    inflected_predictions = predictions
    
    for key,form in inflections.items():
        if form[0] == luText.lower():
            postag = key
            if postag in set(['VB','VBD','VBN','VBP','VBZ']):
                break
    if postag:
        # there can be errors with lemmatization with Pattern for words used as both nouns and verbs like (used, reading), which can lead to invalid inflections
        lemmas = [getAllLemmas(p)['VERB'][0] for p in predictions]
        inflected_predictions = [getInflection(p, tag=postag)[0] for p in lemmas]
    else:
        print('cannot inflect')
        print(luName, luText, inflections)
        
# --------------------------------------------------------------- just to check and correct (if possible) if lemmas do not match predictions
    lemmas = [getAllLemmas(p)['VERB'][0] for p in predictions]
    if set(lemmas) - set(predictions) != set():
        print(set(lemmas) - set(predictions) )
        print(f'{fn}-{luName} ----> {luText},{postag}')
        print(f'predictions:{predictions}')
        print(f'lemmas:     {lemmas}')
        print(f'inflected:  {inflected_predictions}')
# ---------------------------------------------------------------        
#     if len(gd)<=5: print(fn)

    temp_df=pd.DataFrame()
    temp_df['sentence'] = [f'{sentence[:luIndex[0][0]]}<{p}>{sentence[luIndex[0][1]+1:]}' for p in inflected_predictions]
    temp_df['frameName'] = [f'{fn}' for p in predictions]
#     print(frame_info[fn]['definition'])
    temp_df['frame_description'] = [frame_info[fn]['definition'],'','','','','','','','','']
#     temp_df['luName'] = [f'{luName}' for p in predictions]
    temp_df['gold_terms'] = [f'{gd}' for p in predictions]

    temp_df['original_term'] = [f'{luText}' for p in predictions]
    temp_df['match gold'] = [1 if g else 0 for g in match_gold]
    new_df = pd.concat([new_df,temp_df])
#     print(new_df)
#     break
new_df['gold_terms'] = new_df['gold_terms'].apply(lambda x: eval(x))
new_df['gold_terms'] = new_df['gold_terms'].apply(lambda x: ','.join(sorted(x)))

new_df


In [131]:
# new_df.to_csv(f'{data_dir}/manual_evaluation_verbs_pattern.csv', index=False)


### FOR NOUNS

In [ ]:
new_df = pd.DataFrame(columns=['frameName', 'frame_description', 'gold_terms', 'original_term','sentence', 'match gold'])
index_col = 'luIndex'
text_col = 'luText'
name_col = 'luName'

for gd, fn, luName, luText, luIndex, sentence, predictions, match_gold in zip(df[gold_cluster_col], df['frameName'], df[name_col], df[text_col], df[index_col], df['sentence'], df['xlnet_embs'], df['tps']):
#     print(len(predictions), len(match_gold))
    inflections = getAllInflections(luName)
    postag = None
    inflected_predictions = predictions
#     print('--------------------------------')
    for key,form in inflections.items():
        if form[0] == luText.lower():
            postag = key
            if postag in set(['NN', 'NNS', 'NNP', 'NNPS']):
                break
    all_lemmas = {p:getAllLemmas(p) for p in predictions}
    if postag:
        # there can be errors with lemmatization with Pattern for words used as both nouns and verbs like (used, reading), which can lead to invalid inflections
        lemmas = [lemma['NOUN'][0] if 'NOUN' in lemma.keys() else p for p,lemma in all_lemmas.items()]
        inflected_predictions = [getInflection(p, tag=postag)[0] for p in lemmas]
    else:
        print('cannot inflect')
        print(luName, luText, inflections)
# --------------------------------------------------------------- to check and correct (if possible) if lemmas do not match predictions
        
    lemmas = [lemma['NOUN'][0] if 'NOUN' in lemma.keys() else p for p,lemma in all_lemmas.items()]
    if set(lemmas) - set(predictions) != set():
        print('*** discrepancy in lemmas and preds ***')
        print(set(lemmas) - set(predictions) )
        print(f'{fn}-{luName} ----> {luText},{postag}')
        print(f'predictions:{predictions}')
        print(f'lemmas:     {lemmas}')
        print(f'inflected:  {inflected_predictions}')
# ---------------------------------------------------------------        
#     if len(gd)<=5: print(fn)

    temp_df=pd.DataFrame()
    temp_df['sentence'] = [f'{sentence[:luIndex[0][0]]}<{p}>{sentence[luIndex[0][1]+1:]}' for p in inflected_predictions]
    temp_df['frameName'] = [f'{fn}' for p in predictions]
#     print(frame_info[fn]['definition'])
    temp_df['frame_description'] = [frame_info[fn]['definition'],'','','','','','','','','']
#     temp_df['luName'] = [f'{luName}' for p in predictions]
    temp_df['gold_terms'] = [f'{gd}' for p in predictions]

    temp_df['original_term'] = [f'{luText}' for p in predictions]
    temp_df['match gold'] = [1 if g else 0 for g in match_gold]
    new_df = pd.concat([new_df,temp_df])
#     print(new_df)
#     break
new_df['gold_terms'] = new_df['gold_terms'].apply(lambda x: eval(x))
new_df['gold_terms'] = new_df['gold_terms'].apply(lambda x: ','.join(sorted(x)))

new_df


In [136]:
# new_df.to_csv(f'{data_dir}/manual_evaluation_nouns_pattern.csv', index=False)


## Roles

In [319]:
from src.lemmatize_util import pattern_lemma, pattern_parse, nltk_postag
NOUN_POSTAGS = set(['NN', 'NNS','NNP','NNPS'])

def capitalize_rule(seed_word, seed_index, pred):
#     print(seed_word, seed_index, pred)
#     for roles
    if seed_word[0].isupper() and seed_index[0]==0:
        return True
    if seed_word[0].isupper() and nltk_postag(pred) in NOUN_POSTAGS:
        return True
    
    return False


new_df = pd.DataFrame(columns=['frameName', 'frame_description', 'gold_terms', 'feName', 'original_term','sentence', 'match gold'])
index_col = 'feIndex'
text_col = 'feText'
name_col = 'feName'

for gd, fn, luName, luText, luIndex, sentence, predictions, match_gold in zip(df[gold_cluster_col], df['frameName'], df[name_col], df[text_col], df[index_col], df['sentence'], df['xlnet_embs'], df['tps']):
#     print(len(predictions), len(match_gold))
    possible_lemmas = list(itertools.chain(*getAllLemmas(luText).values()))
    inflections = getAllInflections(luText)
    if inflections == {} and possible_lemmas != []:
        inflections = getAllInflections(possible_lemmas[0])
    print('--------------------------------')
    print(luText, ':\t', inflections)
    
    inflected_predictions = predictions
    # match case
#     inflected_predictions = [preds.]
    inflected_predictions = [w.capitalize() if capitalize_rule(luText, luIndex[0], w) else w for w in inflected_predictions]

    postag = None
    need_inflection = False
    possible_postags = set(inflections.keys())
    print(f'possible_postags:{possible_postags}')
    if possible_postags - set(['VB','VBD','VBG','VBN','VBP','VBZ','MD', 'NN', 'NNS', 'NNP', 'NNPS', 'JJ', 'JJR', 'JJS']) == set():
        need_inflection = True
        for key,form in inflections.items():
            if form[0] == luText.lower():
                postag = key            
                break
#     else:
#         need_inflection = False

    all_lemmas = {p:getAllLemmas(p) for p in predictions}
#     print(luText, ':\t', all_lemmas)
    if postag:
        print('\n',postag)
        for k,v in all_lemmas.items(): print(k, ':\t', v)
        # there can be errors with lemmatization with Pattern for words used as both nouns and verbs like (used, reading), which can lead to invalid inflections
        lemmas = [lemma['NOUN'][0] if 'NOUN' in lemma.keys() else p for p,lemma in all_lemmas.items()]
        inflected_predictions = [getInflection(p, tag=postag)[0] for p in lemmas]
        
    else:
        print('cannot inflect')
        print(luText, inflections)
# --------------------------------------------------------------- to check and correct (if possible)
#     lemmas = [lemma['NOUN'][0] if 'NOUN' in lemma.keys() else p for p,lemma in all_lemmas.items()]
#     if set(lemmas) - set(predictions) != set():
#         print('*** discrepancy in lemmas and preds ***')
#         print(set(lemmas) - set(predictions) )
#         print(f'{fn}-{luName} ----> {luText},{postag}')
#         print(f'predictions:{predictions}')
#         print(f'lemmas:     {lemmas}')
#         print(f'inflected:  {inflected_predictions}')
# ----------------------------------------------------------------
    print(inflected_predictions)

    temp_df=pd.DataFrame()
    temp_df['sentence'] = [f'{sentence[:luIndex[0][0]]}<{p}>{sentence[luIndex[0][1]+1:]}' for p in inflected_predictions]
    temp_df['frameName'] = [f'{fn}' for p in predictions]
#     print(frame_info[fn]['definition'])
    temp_df['frame_description'] = [frame_info[fn]['definition'],'','','','','','','','','']
    temp_df['feName'] = [f"{luName}: {frameroles_info[fn]['FEs'][luName]['definition']}" ,'','','','','','','','','']
    temp_df['gold_terms'] = [f'{gd}' for p in predictions]

    temp_df['original_term'] = [f'{luText}' for p in predictions]
    temp_df['match gold'] = [1 if g else 0 for g in match_gold]
#     temp_df['predicted_term'] = inflected_predictions

    new_df = pd.concat([new_df,temp_df])
#     print(new_df)
#     break
new_df['gold_terms'] = new_df['gold_terms'].apply(lambda x: eval(x))
new_df['gold_terms'] = new_df['gold_terms'].apply(lambda x: ','.join(sorted(x)))

new_df
# tinkled, tapering,children


--------------------------------
She :	 {}
possible_postags:set()
cannot inflect
She {}
['He', 'I', 'They', 'You', 'It', 'We', 'Her', 'The', 'Someone', 'Mary']
--------------------------------
they :	 {}
possible_postags:set()
cannot inflect
they {}
['it', 'these', 'those', 'some', 'the', 'them', 'that', 'their', 'we', 'any']
--------------------------------
Bourdieu :	 {}
possible_postags:set()
cannot inflect
Bourdieu {}
['he', 'Miller', 'Smith', 'I', 'Moore', 'Roth', 'Marx', 'Friedman', 'West', 'Shaw']
--------------------------------
land :	 {'NNS': ('lands', 'land'), 'NN': ('land',), 'VBD': ('landed',), 'VBG': ('landing',), 'VBZ': ('lands',), 'VB': ('land',), 'VBP': ('land',)}
possible_postags:{'NNS', 'VBD', 'VBZ', 'NN', 'VBP', 'VB', 'VBG'}

 NN
surface :	 {'NOUN': ('surface',), 'VERB': ('surface',)}
island :	 {'NOUN': ('island',)}
total :	 {'ADJ': ('total',), 'NOUN': ('total',), 'VERB': ('total',)}
present :	 {'ADJ': ('present',), 'NOUN': ('present',), 'VERB': ('present',)}
sea :	

--------------------------------
fiord :	 {}
possible_postags:set()
cannot inflect
fiord {}
['lake', 'river', 'stream', 'beach', 'pond', 'boat', 'bank', 'creek', 'bay', 'rock']
--------------------------------
mom :	 {'NNS': ('moms',), 'NN': ('mom',)}
possible_postags:{'NNS', 'NN'}

 NN
mum :	 {'ADJ': ('mum',), 'NOUN': ('mum',)}
dad :	 {'NOUN': ('dad',)}
mother :	 {'NOUN': ('mother',), 'VERB': ('mother',)}
you :	 {}
daddy :	 {'NOUN': ('daddy',)}
me :	 {}
mama :	 {'NOUN': ('mama',)}
she :	 {}
grandma :	 {'NOUN': ('grandma',)}
her :	 {}
['mum', 'dad', 'mother', 'you', 'daddy', 'me', 'mama', 'she', 'grandma', 'her']
--------------------------------
this :	 {}
possible_postags:set()
cannot inflect
this {}
['it', 'that', 'thing', 'anythe', 'everythe', 'the', 'these', 'them', 'somethe', 'matter']
--------------------------------
window :	 {'NNS': ('windows',), 'NN': ('window',), 'VBD': ('windowed',), 'VBG': ('windowing',), 'VBZ': ('windows',), 'VB': ('window',), 'VBP': ('window',)}
possible_

--------------------------------
conviction :	 {'NNS': ('convictions', 'conviction'), 'NN': ('conviction',)}
possible_postags:{'NNS', 'NN'}

 NN
sentence :	 {'NOUN': ('sentence',), 'VERB': ('sentence',)}
acquittal :	 {'NOUN': ('acquittal',)}
case :	 {'NOUN': ('case',), 'VERB': ('case',)}
trial :	 {'ADJ': ('trial',), 'NOUN': ('trial',), 'VERB': ('trial',)}
charge :	 {'NOUN': ('charge',), 'VERB': ('charge',)}
appeal :	 {'NOUN': ('appeal',), 'VERB': ('appeal',)}
convict :	 {'NOUN': ('convict',), 'VERB': ('convict',)}
innocence :	 {'NOUN': ('innocence',)}
indictment :	 {'NOUN': ('indictment',)}
arrest :	 {'NOUN': ('arrest',), 'VERB': ('arrest',)}
['sentence', 'acquittal', 'case', 'trial', 'charge', 'appeal', 'convict', 'innocence', 'indictment', 'arrest']
--------------------------------
tinkled :	 {'NNS': ('tinkles',), 'NN': ('tinkle',), 'VBD': ('tinkled',), 'VBG': ('tinkling',), 'VBZ': ('tinkles',), 'VB': ('tinkle',), 'VBP': ('tinkle',)}
possible_postags:{'NNS', 'VBD', 'VBZ', 'NN', 'VBP'

,frameName,frame_description,gold_terms,feName,original_term,sentence,match gold
0,Communication_manner,The words in this frame describe Manners of ve...,"adam,adele,agne,alcamo,alexei,all,andrew,angel...",Speaker: Speaker is the sentient entity that p...,She,<He> accepted the glass and muttered a word of...,1
1,Communication_manner,,"adam,adele,agne,alcamo,alexei,all,andrew,angel...",,She,<I> accepted the glass and muttered a word of ...,1
2,Communication_manner,,"adam,adele,agne,alcamo,alexei,all,andrew,angel...",,She,<They> accepted the glass and muttered a word ...,1
3,Communication_manner,,"adam,adele,agne,alcamo,alexei,all,andrew,angel...",,She,<You> accepted the glass and muttered a word o...,1
4,Communication_manner,,"adam,adele,agne,alcamo,alexei,all,andrew,angel...",,She,<It> accepted the glass and muttered a word of...,0
...,...,...,...,...,...,...,...
5,Partiality,,"arbiter,body,both,chairperson,chinese,counsell...",,which,Key data sources for DPRK include defector tes...,0
6,Partiality,,"arbiter,body,both,chairperson,chinese,counsell...",,which,Key data sources for DPRK include defector tes...,0
7,Partiality,,"arbiter,body,both,chairperson,chinese,counsell...",,which,Key data sources for DPRK include defector tes...,0
8,Partiality,,"arbiter,body,both,chairperson,chinese,counsell...",,which,Key data sources for DPRK include defector tes...,1


In [300]:
new_df.to_csv(f'{data_dir}/manual_evaluation_roles_pattern_stopwords4.csv', index=False)
# df_stop = new_df.copy()
# df_stop

In [296]:
# # to avoid reevaluation: take annotations from without stopwords results
# res_dir = 'workdir/manual_evaluation/results'

dataset, gold_ds  = 'roles_st', 'gold_dataset_roles.pkl'
# dataset, gold_ds  = 'roles_st_stopwords4', 'gold_dataset_roles.pkl'
# results = pd.read_csv(f'{res_dir}/{dataset}/xlnet_embs/results.tsv', sep='\t') 

# results['predicted_term'] = new_df['predicted_term'].tolist()
results['predicted_term_stopwords4'] = df_stop['predicted_term'].tolist()

results



,frameName,frame_description,gold_terms,feName,original term,sentence,match gold,does not fit context,fit context NOT frame,fit context AND frame,predicted_term,predicted_term_stopwords4
0,Communication_manner,The words in this frame describe Manners of ve...,"adam,adele,agne,alcamo,alexei,all,andrew,angel...",Speaker: Speaker is the sentient entity that p...,She,<He> accepted the glass and muttered a word of...,1,NaN,NaN,3.0,He,He
1,Communication_manner,NaN,"adam,adele,agne,alcamo,alexei,all,andrew,angel...",NaN,She,<I> accepted the glass and muttered a word of ...,1,NaN,NaN,3.0,I,I
2,Communication_manner,NaN,"adam,adele,agne,alcamo,alexei,all,andrew,angel...",NaN,She,<They> accepted the glass and muttered a word ...,1,NaN,NaN,3.0,They,They
3,Communication_manner,NaN,"adam,adele,agne,alcamo,alexei,all,andrew,angel...",NaN,She,<You> accepted the glass and muttered a word o...,1,NaN,NaN,3.0,You,You
4,Communication_manner,NaN,"adam,adele,agne,alcamo,alexei,all,andrew,angel...",NaN,She,<It> accepted the glass and muttered a word of...,0,1.0,NaN,NaN,It,It
...,...,...,...,...,...,...,...,...,...,...,...,...
495,Partiality,NaN,"arbiter,body,both,chairperson,chinese,counsell...",NaN,which,Key data sources for DPRK include defector tes...,0,1.0,NaN,NaN,the,they
496,Partiality,NaN,"arbiter,body,both,chairperson,chinese,counsell...",NaN,which,Key data sources for DPRK include defector tes...,0,1.0,NaN,NaN,where,what
497,Partiality,NaN,"arbiter,body,both,chairperson,chinese,counsell...",NaN,which,Key data sources for DPRK include defector tes...,0,1.0,NaN,NaN,whose,these
498,Partiality,NaN,"arbiter,body,both,chairperson,chinese,counsell...",NaN,which,Key data sources for DPRK include defector tes...,1,1.0,NaN,NaN,it,some


In [314]:
# results.loc[results['predicted_term_stopwords4']!=results['predicted_term']]

# dataset, gold_ds  = 'roles_st_stopwords4', 'gold_dataset_roles.pkl'
# results.to_csv(f'{res_dir}/{dataset}/xlnet_embs/results_helpfile.tsv', sep='\t', index=False)

In [313]:
# new_df.loc[new_df['frameName']=='Non-gradable_proximity']

## Evaluation

In [320]:
res_dir = 'workdir/manual_evaluation/results'

dataset, gold_ds  = 'verb_preds_st', 'gold_dataset_verbs.pkl'
dataset, gold_ds  = 'noun_preds_st', 'gold_dataset_nouns.pkl'
dataset, gold_ds  = 'roles_st', 'gold_dataset_roles.pkl'
# dataset, gold_ds  = 'roles_st_stopwords4', 'gold_dataset_roles_stopwords4.pkl'

gold_df = df#[:10]
gold_df.to_pickle(f'{res_dir}/{gold_ds}')
save_dir_path = f'{res_dir}/{dataset}/xlnet_embs_gold'


tps, aps, gps = gold_df['tps'], gold_df['aps'], gold_df['gps']

print('Saving resutls...')
with open(os.path.join(save_dir_path, 'tps.pkl'), 'wb') as f:
    pickle.dump(tps, f)

with open(os.path.join(save_dir_path, 'aps.pkl'), 'wb') as f:
    pickle.dump(aps, f)

with open(os.path.join(save_dir_path, 'gps.pkl'), 'wb') as f:
    pickle.dump(gps, f)   
    
# tps, aps, gps    

Saving resutls...


In [323]:
import os
import csv
import pickle
import pandas as pd

res_dir = 'workdir/manual_evaluation/results'

dataset, gold_ds  = 'verb_preds_st', 'gold_dataset_verbs.pkl'
dataset, gold_ds  = 'noun_preds_st', 'gold_dataset_nouns.pkl'
dataset, gold_ds  = 'roles_st', 'gold_dataset_roles.pkl'
dataset, gold_ds  = 'roles_st_stopwords4', 'gold_dataset_roles_stopwords4.pkl'


exp_path = f'{res_dir}/{dataset}/xlnet_embs'
save_dir_path = f'{res_dir}/{dataset}/xlnet_embs'

temp_df = pd.read_csv(f'{exp_path}/results.tsv', sep='\t', quoting=csv.QUOTE_NONE)
temp_df.fillna(0, inplace=True)
#
temp_df['match gold'] = temp_df['match gold'].apply(lambda x: 0 if x==-1 else x)
# 
L = len(temp_df)//10

gold_df = pd.read_pickle(f'{res_dir}/{gold_ds}')[:L]

print(len(gold_df), len(temp_df))

k =10

column_map ={'does not fit context':'NC', 
          'fit context NOT frame': 'C', 
          'fit context AND frame': 'CF', 
          'match gold': 'G'}

for column, subdir in column_map.items():
#     if subdir!='G': continue
    print(column, subdir)
    save_dir_path = f'{exp_path}/{subdir}'
    if not os.path.exists(save_dir_path):
        os.mkdir(save_dir_path)    
        
    tps, aps, gps =[],[],[]
    for i in range(0, len(gold_df)):
    #     print(i, i*k, i*k+k)
    #     print(gold_df['gold_cluster'].iloc[i], len(gold_df['gold_cluster'].iloc[i]))
    #     fn, original_term, substituted_term, sentence = temp_df['frameName'].iloc[i],temp_df['original_term'].iloc[i],temp_df['substituted_term'].iloc[i],temp_df['sentence'].iloc[i]
    #     sentence = sentence.replace(f'<{substituted_term}>', original_term)
        res = temp_df[column][i*k:i*k+k]
        _tps, _gps = [True if r else False for r in res], max(0, len(res))
        _aps = []
        for j in range(k):
            if j < len(res):
                _aps.append(j+1)
            else:
                _aps.append(len(res))
        aps.append(_aps)
        gps.append(_gps)
        tps.append(_tps)

    tps, gps, aps = tuple(tps), tuple(gps), tuple(aps)    

    print('Saving resutls...')
    with open(os.path.join(save_dir_path, 'tps.pkl'), 'wb') as f:
        pickle.dump(tps, f)

    with open(os.path.join(save_dir_path, 'aps.pkl'), 'wb') as f:
        pickle.dump(aps, f)

    with open(os.path.join(save_dir_path, 'gps.pkl'), 'wb') as f:
        pickle.dump(gps, f)    

50 500
does not fit context NC
Saving resutls...
fit context NOT frame C
Saving resutls...
fit context AND frame CF
Saving resutls...
match gold G
Saving resutls...


In [324]:
import multiprocessing
import numpy as np
from tqdm import tqdm
def precision_recall_curve(tps, aps, gps, n_jobs):
    with multiprocessing.Pool(n_jobs) as pool:
        tps = pool.map(np.cumsum, tqdm(tps), chunksize=800)
    
    tps, aps, gps = (np.array(e).sum(axis=0) for e in (tps, aps, gps)) # aps already contains the cumulative sum
    # gps contains the number of all gold results
    return tps / aps, tps / gps



def precision_recall_curve_hard(tps, gps, n_jobs):
#     print(f'tps: {tps}')
#     print(f'gps: {gps}')
    aps = np.array(list(range(1, len(tps[0]) + 1)))
    aps = aps * len(tps)
#     print(f'aps: {aps}') 
    with multiprocessing.Pool(n_jobs) as pool:
        tps = pool.map(np.cumsum, tqdm(tps), chunksize=800)
        
#     print(f'cusum tps:') 
#     for i in range(0,len(tps)):
#         print(tps[i])
    tps, gps = (np.array(e).sum(axis=0) for e in (tps, gps)) 
#     print(f'axis sum tps: {tps}')
    
    
#     print(f'gps: {gps}')
#     print(f'tps/aps: {tps / aps}')
#     print(f'tps/gps: {tps / gps}')
    
    return tps / aps, tps / gps

def precision_at_level_hard(tps, levels):
    result = []
    
    for lev in levels:
        all_t = sum([sum(t[:lev]) for t in tps])
        ap_t = lev * len(tps)
        result.append(all_t / ap_t)
#         print(f'P@{lev}: {all_t} {ap_t}')
    return result

def create_precision_recall_curve(annots, exp_name=None, output_file_path=None):
    tps, aps, gps = annots
    #precision, recall = precision_recall_curve(tps, aps, gps, n_jobs=6)
    precision, recall = precision_recall_curve_hard(tps, gps, n_jobs=6)
    metrics = pd.DataFrame.from_dict({'precision' : precision, 'recall' : recall})
    if output_file_path is not None:
        metrics.to_csv(output_file_path)
    return metrics

def calc_MAP_at_k(tps, gps, k=None):
    assert len(tps) == len(gps)
    
    if k is None:
        k = len(tps[0])
        
    if gps is None:
        gps = [len(tps[0])] * len(tps)
        
    prec_div = np.array(range(1, len(tps[0]) + 1))
#     print(prec_div)
    map_at_k = 0.
    for tp, gp in zip(tps, gps): 
        
        div = min(gp, k)
#         print(div)
        ap = ((np.cumsum(tp) /  prec_div) * np.array(tp)).sum() / div
        map_at_k += ap
#         print(tp, gp, ap, map_at_k)
    return map_at_k / len(tps)

res_df = pd.DataFrame(columns = ['p@1', 'p@3', 'p@5', 'p@10'], index=columns_map.keys())
for column, subdir in column_map.items():
    print(column, subdir)
    save_dir_path = f'{exp_path}/{subdir}'

    annots = load_tps_aps_gps(save_dir_path)

    levels = [1,3, 5,10]
    k=10
#     exp_name = 'xlm_embs_manual'
    annots_df = pd.DataFrame({'tps' : annots[0], 'aps' : annots[1], 'gps' : annots[2]})
    annots_df = annots_df[annots_df.gps != 0]
    # print(len(annots_df))
    annots = annots_df.tps.tolist(), annots_df.aps.tolist(), annots_df.gps.tolist()

    curve = create_precision_recall_curve(annots, exp_name=None, 
                                          output_file_path=None)
    tps, aps, gps = annots

    precs = precision_at_level_hard(tps, levels=levels)
    # mean_av_prec = calc_MAP_at_k(tps, gps, k)


    metrics = {}
    metrics['precisions_at_level'] = {str(lev) : prec for lev, prec in zip(levels, precs)}
    # metrics['map'] = mean_av_prec
    res = [prec for lev, prec in zip(levels, precs)]
    # res.append(mean_av_prec)
    print(res)
    res_df.loc[column] = res
#     metrics

res_df

does not fit context NC


100%|██████████| 50/50 [00:00<00:00, 28297.83it/s]


[0.22, 0.21333333333333335, 0.236, 0.268]
fit context NOT frame C


100%|██████████| 50/50 [00:00<00:00, 27338.70it/s]


[0.12, 0.12666666666666668, 0.132, 0.15]
fit context AND frame CF


100%|██████████| 50/50 [00:00<00:00, 18315.74it/s]


[0.64, 0.6533333333333333, 0.636, 0.588]
match gold G


100%|██████████| 50/50 [00:00<00:00, 13400.33it/s]


[0.42, 0.43333333333333335, 0.42, 0.35]


,p@1,p@3,p@5,p@10
does not fit context,0.22,0.213333,0.236,0.268
fit context NOT frame,0.12,0.126667,0.132,0.15
fit context AND frame,0.64,0.653333,0.636,0.588
match gold,0.42,0.433333,0.42,0.35


In [105]:
load_dir_path = '../workdir/manual_results/paper_preds_st/xlnet_embs_gold'
load_dir_path = '../workdir/manual_results/paper_roles_st/xlnet_embs_gold'

annots = load_tps_aps_gps(load_dir_path)

annots_df = pd.DataFrame({'tps' : annots[0], 'aps' : annots[1], 'gps' : annots[2]})
annots_df = annots_df[annots_df.gps != 0]
# print(len(annots_df))
annots = annots_df.tps.tolist(), annots_df.aps.tolist(), annots_df.gps.tolist()

curve = create_precision_recall_curve(annots, exp_name=None, 
                                      output_file_path=None)
tps, aps, gps = annots

precs = precision_at_level_hard(tps, levels=levels)
mean_av_prec = calc_MAP_at_k(tps, gps, k)


metrics = {}
metrics['precisions_at_level'] = {str(lev) : prec for lev, prec in zip(levels, precs)}
metrics['map'] = mean_av_prec
res = [prec for lev, prec in zip(levels, precs)]
res.append(mean_av_prec)
print(res)

metrics



100%|██████████| 50/50 [00:00<00:00, 38879.35it/s]

[0.46, 0.42, 0.416, 0.338, 0.2625521164021164]


{'precisions_at_level': {'1': 0.46, '3': 0.42, '5': 0.416, '10': 0.338},
 'map': 0.2625521164021164}

In [84]:
from nltk.corpus import wordnet
for result in wordnet.synsets('skill'):
#     print(result.name(), result.definition(), result.examples())
    print(result.name())


skill.n.01
skill.n.02


## stats


In [21]:
dfv

,frameName,luName,luText,luIndex,sentence,gold_cluster,fasttext,dt59g,baladd,blc,xlnet_embs,tps,aps,gps
59294,Leadership,preside,presiding,"[(148, 156)]",In an attempt to continue funding its advanced...,"[govern, spearhead, preside, head, administer,...","[superintend, judge, appoint, chairman, office...","[officiate, dissent, live, founder, surrogate,...","[officiate, deliberate, adjudicate, mull, pore...","[reign, rule, hold, lead, helm, govern, head, ...","[reign, oversee, supervise, rule, govern, prev...","(True, False, False, True, True, False, False,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)",10
73370,Placing,drape,draped,"[(65, 70)]",Schools and most offices closed ; taxis flew b...,"[insert, shower, inject, cram, billet, implant...","[swathe, adorn, bedeck, festoon, draping, unfu...","[breathe, reason, bewitch, scream, originate, ...","[festoon, swathe, hang, bedeck, plaster, furl,...","[hang, wave, fly, stream, sling, rip, unfold, ...","[hang, wrapped, adorn, spray, paint, display, ...","(True, False, False, False, False, False, Fals...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)",65
61033,Perception_active,watch,watching,"[(18, 25)]",The companies are watching carefully lest othe...,"[gape, gawk, goggle, peek, peep, feel, peer, s...","[see, enjoy, observe, view, listen, witness, c...","[bike, hike, camp, boat, canoe, kayak, fishes,...","[see, listen, observe, eye, scrutinize, witnes...","[monitor, observe, look, study, guard, listen,...","[observe, listen, view, monitor, wait, see, re...","(True, True, True, False, False, False, False,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)",25
59222,Conduct,misbehave,misbehave,"[(9, 17)]",Children misbehave when they are discouraged a...,"[act, conduct, misbehave, behave]","[behave, scold, punish, spank, mistreat, chast...","[spank, disobey, try, violate, shoplift, find,...","[behave, over-react, overeat, mistreat, undera...","[cry, learn, succeed, play, act, suffer, work,...","[suffer, fail, learn, behave, succeed, disobey...","(False, False, False, True, False, False, Fals...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)",3
66563,Filling,inject,injected,"[(23, 30)]",Some of the males were injected with morphine ...,"[butter, asphalt, surface, anoint, tile, adorn...","[infuse, dose, implant, inoculate, insert, syr...","[add, ingest, inhale, purify, contaminate, pow...","[dose, pump, resuspend, infuse, implant, trans...","[insert, administer, treat, provide, pump, sta...","[insert, pump, treat, spray, administer, infec...","(False, True, False, True, False, False, False...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)",58
24890,Attaching,solder,soldered,"[(26, 33)]",Where the mains leads are soldered to the outp...,"[moor, cinch, agglutinate, solder, shackle, we...","[unsolder, epoxy, braze, weld, socket, spot-we...","[re-form, gild, interest, antique, punch, prec...","[weld, braze, fasten, re-attach, tension, rive...","[sell, deliver, connect, ship, wire, send, use...","[sell, fit, rout, connect, wire, supply, mount...","(False, False, False, True, True, False, True,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)",50
24203,Execute_plan,implement,implemented,"[(111, 121)]",He later produced the Blackburn Report on the ...,"[institute, implement]","[develop, enact, introduce, institute, devise,...","[advertise, license, learn, authorise, prefer,...","[enact, introduce, enforce, deploy, adopt, und...","[enact, adopt, enforce, introduce, institute, ...","[institute, enact, enforce, adopt, execute, pr...","(True, False, False, False, False, False, Fals...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)",1
67096,Coming_to_believe,guess,guess,"[(66, 70)]",I felt angry and humiliated at the time of the...,"[speculate, surmise, deduce, determine, realiz...","[suppose, think, know, figure, presume, say, w...","[think, believe, say, know, realize, realise, ...","[surmise, suppose, presume, reckon, imagine, t...","[imagine, estimate, judge, wonder, tell, pictu...","[suppose, imagine, think, speculate, assume, c...","(False, False, False, True, False, False

In [27]:
import pandas as pd
df = pd.DataFrame(columns=["characteristics", "verbs","nouns","roles"])
# dfv = pd.read_csv("workdir/manual_evaluation/data/manual_evaluation_verbs_pattern.csv")
# dfn = pd.read_csv("workdir/manual_evaluation/data/manual_evaluation_nouns_pattern.csv")
# dfr = pd.read_csv("workdir/manual_evaluation/data/manual_evaluation_roles_pattern.csv")
# dfv["id"] = dfv["frameName"]+"#"+dfv["luName"]
# dfn["id"] = dfn["frameName"]+"#"+dfn["luName"]
# dfr["id"] = dfr["frameName"]+"#"+dfr["feName"]



In [37]:
dfr

,frameName,feName,feText,feIndex,sentence,gold_cluster_patternlemmatized,w2v,dtwiki,baladd,blc,xlnet_embs,tps,aps,gps,id
118588,Communication_manner,Speaker,She,"[(0, 2)]",She accepted the glass and muttered a word of ...,"[i, we, he, she, daisy, luke, jimmy, vlito, al...","[her, herself, he, woman, rivadineira, jana_bo...","[he, they, my, whose, you, his/her, it, we, wh...","[he, they, who, tashigi, we, i, urahara, rajin...","[he, i, her, rachel, they, sara, rise, sarah, ...","[he, i, they, you, it, we, her, the, someone, ...","(True, True, True, True, False, True, True, Fa...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)",209,Communication_manner#Speaker
146740,Mental_property,Behavior,they,"[(87, 90)]",He would think judges should retain the power ...,"[investment, wit, behavior, tend, script, rapp...","[we, them, their, do, not, have, themselve, if...","[it, he, whose, she, my, his/her, we, you, ""ou...","[we, he, she, it, i, you, itll, who, mightest,...","[it, them, those, these, decision, there, orde...","[it, these, those, some, the, them, that, thei...","(True, False, False, False, False, False, True...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)",121,Mental_property#Behavior
122067,Willingness,Cognizer,Bourdieu,"[(36, 43)]",In much of his treatment of Kabylia Bourdieu i...,"[libya, he, it, republican, god, government, w...",[],"[merton, millennium, rubenstein, sewer, beaucl...","[bakhtin, haberma, derrida, irigaray, mouffe, ...","[he, bo, she, carr, bacon, they, hunt, oliver,...","[he, miller, smith, i, moore, roth, marx, frie...","(True, False, False, True, False, False, False...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)",20,Willingness#Cognizer
118445,Natural_features,Locale,land,"[(63, 66)]","Under a 99-year lease , the US acquired about ...","[mountain, ridge, gulf, island, beach, atoll, ...","[farmland, acre, undeveloped_land, property, t...","[area, property, farmland, estate, territory, ...","[nwdep, marshland, dumbiedyke, cropland, farml...","[territory, property, grind, island, country, ...","[surface, island, total, present, sea, shore, ...","(False, True, False, False, True, False, False...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)",118,Natural_features#Locale
133127,Manipulation,Entity,Madeleine,"[(13, 21)]",She clutched Madeleine around the waist .,"[me, them, it, her, him, brückner, jimmy, u, h...","[proust_madeleine, proustian, grand_marnier_so...","[pastry, dessert, cake, l'amour, cloth, bacon,...","[napkin, bonce, madeline, chantel, florrie, va...","[him, it, herself, them, maddie, her, me, spen...","[her, sophie, him, herself, julia, me, louise,...","(True, False, True, True, False, True, False, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)",37,Manipulation#Entity
153979,Judgment,Cognizer,he,"[(12, 13)]",And why had he been so angry and contemptuous ...,"[i, voter, parent, we, prisoner, roddick, he, ...","[him, hi, she, himself, nobody, i, it, never, ...","[she, they, whose, my, it, his/her, you, we, ""...","[she, they, tashigi, who, urahara, juubei, war...","[she, they, it, i, you, hi, we, who, that, eve...","[she, i, they, it, you, that, we, him, everyon...","(True, True, True, True, True, True, True, Tru...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)",131,Judgment#Cognizer
49781,Prison,Operator,town,"[(24, 27)]",The third storey of the town gaol was pierced ...,"[military, swedish, community, scottish, repub...","[village, hamlet, city, townspeople, tiny_haml...","[city, village, settlement, castle, area, comm...","[city, county, warrenpoint, village, kilmainha...","[city, borough, county, village, new, old, law...","[city, village, old, county, local, london, di...","(True, False, False, True, True, False, False,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)",14,Prison#Operator
48407,Location_of_light,Figure,black,"[(42, 46)]",But in the moonlight now it looks a shiny blac...,"[day, hotel, it, tear, that, water, which, she...","[white, responded_letterman_how, crypt_inscrib...","[white, red, blue, yellow, green, brown, dark,...","[white, sky-blue, silver-grey,

In [29]:
# df.loc[len(df)] = ["frames", dfv["frameName"].nunique(), dfn["frameName"].nunique(), dfr["frameName"].nunique()]
# df.loc[len(df)] = ["lu/framerole", dfv["id"].nunique(), dfn["id"].nunique(), dfr["id"].nunique()]
df.loc[len(df)] = ["sentence", dfv["sentence"].nunique(), dfn["sentence"].nunique(), dfr["sentence"].nunique()]
df

,characteristics,verbs,nouns,roles
0,frames,41,43,44
1,lu/framerole,50,50,47
2,sentence,50,50,50


## to verify role extraction

In [30]:
import pandas as pd
df = pd.read_csv('data_files/fn_data_v1.csv')
df = df.drop_duplicates(subset=['annoID'])
df['feTotal'] = df['frameElements'].apply(lambda x: len(eval(x)))
df['feLength'] = df['frameElements'].apply(lambda x: [len(x[0].split(' ')) for x in eval(x)])
df['feLengthMax'] = df['feLength'].apply(lambda x: max(x) if x else 0)

df

,frameID,frameName,sentence,luName,luID,luText,luPOS,luIndex,frameElements,sentenceID,annoID,file,feTotal,feLength,feLengthMax
0,1892,Work,Work Advances at Iranian Uranium Enrichment Site,work,14001,Work,n,"[(0, 3)]","[('null', 'INI', 'Goal'), ('null', 'INI', 'Age...",4100865,6542766,NTI__workAdvances.xml,2,"[1, 1]",1
1,620,Processing_materials,Work Advances at Iranian Uranium Enrichment Site,enrichment,10646,Enrichment,n,"[(33, 42)]","[('Uranium', (25, 31), 'Material')]",4100865,6542769,NTI__workAdvances.xml,1,[1],1
2,196,Locale_by_use,Work Advances at Iranian Uranium Enrichment Site,site,11550,Site,n,"[(44, 47)]","[('Iranian', (17, 23), 'Container_possessor'),...",4100865,6542770,NTI__workAdvances.xml,3,"[1, 2, 1]",2
3,1340,Progression,Work Advances at Iranian Uranium Enrichment Site,advance,11644,Advances,v,"[(5, 12)]","[('Work', (0, 3), 'Entity'), ('at Iranian Uran...",4100865,6543165,NTI__workAdvances.xml,2,"[1, 5]",5
4,199,Locative_relation,Work Advances at Iranian Uranium Enrichment Site,at,6735,at,prep,"[(14, 15)]","[('Work Advances', (0, 12), 'Figure'), ('Irani...",4100865,6543403,NTI__workAdvances.xml,2,"[2, 4]",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230905,203,Roadways,"THE BARD was striding along the Boulevard , cl...",boulevard,3797,Boulevard,n,"[(32, 40)]","[('Boulevard', (32, 40), 'Roadway')]",505383,616556,lu3797.xml,1,[1],1
230906,203,Roadways,"In the main boulevard of Havana , they have ar...",boulevard,3797,boulevard,n,"[(12, 20)]","[('main', (7, 10), 'Characteristic'), ('boulev...",505409,616608,lu3797.xml,3,"[1, 1, 2]",2
230907,203,Roadways,He left the hospital and began to walk slowly ...,boulevard,3797,boulevard,n,"[(56, 64)]","[('boulevard', (56, 64), 'Roadway'), ('leading...",505414,616618,lu3797.xml,2,"[1, 7]",7
230908,203,Roadways,"Not for him the boulevards of Hollywood , nor ...",boulevard,3797,boulevards,n,"[(16, 25)]","[('of Hollywood', (27, 38), 'Relative_location...",505418,616626,lu3797.xml,2,"[2, 1]",2


In [35]:
dfs = df.loc[df['frameName']=='Mental_property']

dfs.loc[dfs['sentence'].apply(lambda x: x.startswith("He would think judges"))]

,frameID,frameName,sentence,luName,luID,luText,luPOS,luIndex,frameElements,sentenceID,annoID,file,feTotal,feLength,feLengthMax
186852,29,Mental_property,He would think judges should retain the power ...,ill-advised,402,ill-advised,a,"[(108, 118)]","[('they', (87, 90), 'Behavior'), ('especially'...",391882,426192,lu402.xml,2,"[1, 1]",1


In [36]:
"He would think judges should retain the power to overrule past judicial decisions , if <it> were especially ill-advised , even when they were clear enough to provide crisp guidance to litigants ."[87:91]

'<it>'

### Upperbound

In [168]:
import random
random.seed(42)
from src.postprocessing import remove_seedword
from src.postprocessing import lemmatize
import json
def upperbound_baseline(seed_words, gold_clusters, 
                        save_dir_path,
                        test_indexes_path=None,
                        k=50):
    
    assert len(seed_words) == len(gold_clusters)
    
    
    
    if test_indexes_path is not None:
        df = pd.DataFrame()
        df['gold_clusters'] = gold_clusters
        df['seed_words'] = seed_words
        with open(test_indexes_path, 'r') as f:
            test_indexes = json.load(f)
            if len(gold_clusters)!= len(test_indexes):
                        df = df.loc[test_indexes].copy().reset_index(drop=True)    
        seed_words = df['seed_words']
        gold_clusters = df['gold_clusters']
                
    predictions = []
    
    seed_words = [w.lower() for w in seed_words]
#     seed_words = lemmatize(seed_words)
    gold_clusters_preds = remove_seedword(seed_words, gold_clusters)
    
    for w, gd in zip(seed_words, gold_clusters_preds):
        
        G = len(gd)
        if G < k:
            preds = random.sample(gd, G)
            for i in range(k-G):
                preds.append("NO")
        else:
            preds = random.sample(gd, k)
#         print(w)
#         print(gd)
#         print(preds)    
        predictions.append(preds)
    
    print('Saving final predictions...', len(predictions))
    
    if not os.path.exists(save_dir_path):
        os.mkdir(save_dir_path)
        
    with open(os.path.join(save_dir_path, 'final_predictions.pkl'), 'wb') as f:
        pickle.dump(predictions, f)
        
    calculate_true_positives(predictions, 
                         gold_clusters, 
                         save_dir_path,
                         k=k)
    
    return predictions



In [87]:
import pandas as pd
dfg = pd.read_pickle("workdir/data/swn_gold_dataset.pkl")
dfg.head(5)

,frameID,frameName,luName,luID,luText,luIndex,sentence,gold_cluster,gold_cluster_processed
0,1892,Work,work,14001,Work,"[(0, 3)]",Work Advances at Iranian Uranium Enrichment Site,[work],[work]
1,620,Processing_materials,enrichment,10646,Enrichment,"[(33, 42)]",Work Advances at Iranian Uranium Enrichment Site,"[development, weaponization, reprocessing, enr...","[development, weaponization, reprocessing, enr..."
2,196,Locale_by_use,site,11550,Site,"[(44, 47)]",Work Advances at Iranian Uranium Enrichment Site,"[hedge, cemetery, village, countryside, center...","[hedge, cemetery, village, countryside, center..."
3,189,Quantified_mass,thousands,3328,thousands,"[(43, 51)]",Iran has begun preliminary work to install tho...,"[mountain, pinch, handful, ounce, mite, pile, ...","[mountain, pinch, handful, ounce, mite, pile, ..."
4,1321,Substance,uranium,12169,uranium,"[(75, 81)]",Iran has begun preliminary work to install tho...,"[juice, gas, metal, oil, glass, paper, liquid,...","[juice, gas, metal, oil, glass, paper, liquid,..."


In [171]:
save_dir_path = "workdir/upperbound_results/paper_preds_st"
test_indexes_path = 'data_files/swv_gold_dataset_test_split.json'
with open(test_indexes_path, 'r') as f:
    test_indexes = json.load(f)
dfg = pd.read_pickle("workdir/data/swv_gold_dataset.pkl")

# save_dir_path = "workdir/upperbound_results/paper_nouns_st"
# test_indexes_path = 'data_files/swn_gold_dataset_test_split.json'
# with open(test_indexes_path, 'r') as f:
#     test_indexes = json.load(f)
# dfg = pd.read_pickle("workdir/data/swn_gold_dataset.pkl")

final_preds = upperbound_baseline(dfg['luName'], dfg['gold_cluster_processed'],
                                 save_dir_path,
                                test_indexes_path=test_indexes_path,
                                 k=50)



# dfg = pd.read_pickle("workdir/data/swr_gold_dataset.pkl")
# save_dir_path = "workdir/upperbound_results/paper_roles_st"
# test_indexes_path = 'data_files/swr_gold_dataset_test_split.json'

# final_preds = upperbound_baseline(dfg['feText'], dfg['gold_cluster_patternlemmatized'],
#                                  save_dir_path,
#                                 test_indexes_path=test_indexes_path,
#                                  k=50)


100%|██████████| 75584/75584 [00:01<00:00, 74039.54it/s] 


Saving final predictions... 75584
Calculating true positives


100%|██████████| 75584/75584 [00:16<00:00, 4526.14it/s] 


Saving resutls...
Done.


In [160]:
dfg = dfg.loc[test_indexes].copy().reset_index(drop=True)   

dfg['final_preds'] = final_preds

In [172]:
import src
!python -m src.run_evaluate \
--results_path=workdir/upperbound_results
# --exp_names='paper_preds_st'


paper_roles_st  ########################
181094
100%|███████████████████████████████| 181094/181094 [00:01<00:00, 150989.27it/s]
Precision at levels:
(1, 1.0)
(5, 0.9711564160049477)
(10, 0.9347791754558407)
MAP:  1.0
################################
paper_preds_st  ########################
73204
100%|█████████████████████████████████| 73204/73204 [00:00<00:00, 142269.66it/s]
Precision at levels:
(1, 1.0)
(5, 0.9041063329872685)
(10, 0.7956969564504672)
MAP:  1.0
################################
paper_nouns_st  ########################
70056
100%|█████████████████████████████████| 70056/70056 [00:00<00:00, 151317.10it/s]
Precision at levels:
(1, 1.0)
(5, 0.9179627726390316)
(10, 0.8358427543679342)
MAP:  1.0
################################


In [152]:
import pandas as pd
def load_tps_aps_gps(load_dir_path):
    with open(os.path.join(load_dir_path, 'tps.pkl'), 'rb') as f:
        tps = pickle.load(f)

    with open(os.path.join(load_dir_path, 'aps.pkl'), 'rb') as f:
        aps = pickle.load(f)

    with open(os.path.join(load_dir_path, 'gps.pkl'), 'rb') as f:
        gps = pickle.load(f)
        
    return tps, aps, gps

save_dir_path = "workdir/upperbound_results/paper_nouns_st"

annots = load_tps_aps_gps(save_dir_path)
annots_df = pd.DataFrame({'tps' : annots[0], 'aps' : annots[1], 'gps' : annots[2]})
annots_df = annots_df[annots_df.gps != 0]
annots = annots_df.tps.tolist(), annots_df.aps.tolist(), annots_df.gps.tolist()
tps, aps, gps = annots    
tps[0], gps[0]

([True,
  True,
  True,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False],
 3)

In [147]:
save_dir_path = "workdir/results/test-paper_nouns_st_pattern_stopwords_nounfilter/gie_swn_test_semiPURExlnet_embs_swnhypers"

annots = load_tps_aps_gps(save_dir_path)
annots_df = pd.DataFrame({'tps' : annots[0], 'aps' : annots[1], 'gps' : annots[2]})
annots_df = annots_df[annots_df.gps != 0]
annots = annots_df.tps.tolist(), annots_df.aps.tolist(), annots_df.gps.tolist()
tps, aps, gps = annots    
tps[0], gps[0]

([True,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False],
 3)

In [149]:
import numpy as np
import itertools

def calc_MAP_at_k(tps, gps, k=None):
    assert len(tps) == len(gps)
    
    if k is None:
        k = len(tps[0])
        
    if gps is None:
        gps = [len(tps[0])] * len(tps)
        
    prec_div = np.array(range(1, len(tps[0]) + 1))
#     print(prec_div)
    map_at_k = 0.
    for tp, gp in zip(tps, gps): 
        div = min(gp, k)
#         print(div)
        ap = ((np.cumsum(tp) /  prec_div) * np.array(tp)).sum() / div
        map_at_k += ap
#         print(tp)
        if ap > 1.0000:
            print(ap)
    print(map_at_k, len(tps))
    return map_at_k / len(tps)


calc_MAP_at_k(tps, gps, k =50)

11983.147589033153 70056


0.17105098191494167

In [150]:
i =0
for i in range(len(tps)):
#     print(tps[i])
    prec_div = np.array(range(1, len(tps[i]) + 1))
    # print(prec_div)
    div = min(gps[i], 50)
    # print(div)
    ap = ((np.cumsum(tps[i]) /  prec_div) * np.array(tps[i])).sum() / div
    if ap != 1.0000:
        print(i)
        print(tps[i])
        print(ap)
        break

0
[True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
0.3333333333333333


In [153]:
i = 2
print(tps[i])
prec_div = np.array(range(1, len(tps[i]) + 1))
print(prec_div)
div = min(gps[i], 50)
print(div)
ap = ((np.cumsum(tps[i]) /  prec_div) * np.array(tps[i])).sum() / div
if ap != 1.0000:
    print(ap)

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50]
43
1.0232558139534884


In [154]:
np.cumsum(tps[i])

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 44, 44, 44, 44, 44, 44])

In [143]:
np.cumsum(tps[i])/  prec_div

array([1.        , 1.        , 1.        , 0.75      , 0.6       ,
       0.5       , 0.42857143, 0.375     , 0.33333333, 0.3       ,
       0.27272727, 0.25      , 0.23076923, 0.21428571, 0.2       ,
       0.1875    , 0.17647059, 0.16666667, 0.15789474, 0.15      ,
       0.14285714, 0.13636364, 0.13043478, 0.125     , 0.12      ,
       0.11538462, 0.11111111, 0.10714286, 0.10344828, 0.1       ,
       0.09677419, 0.09375   , 0.09090909, 0.08823529, 0.08571429,
       0.08333333, 0.08108108, 0.07894737, 0.07692308, 0.075     ,
       0.07317073, 0.07142857, 0.06976744, 0.06818182, 0.06666667,
       0.06521739, 0.06382979, 0.0625    , 0.06122449, 0.06      ])

In [144]:
np.cumsum(tps[i])/  prec_div * np.array(tps[i])

array([1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [145]:
((np.cumsum(tps[i]) /  prec_div) * np.array(tps[i])).sum()

3.0

In [164]:
dfg['G'] = dfg['gold_cluster_processed'].apply(lambda x: len(x))
dfg['P'] = dfg['final_preds'].apply(lambda x: len([p for p in x if p!="NO"]))

In [165]:
dfg

,frameID,frameName,luName,luID,luText,luIndex,sentence,gold_cluster,gold_cluster_processed,final_preds,G,P
0,1892,Work,work,14001,Work,"[(0, 3)]",Work Advances at Iranian Uranium Enrichment Site,[work],[work],"[NO, NO, NO, NO, NO, NO, NO, NO, NO, NO, NO, N...",1,0
1,620,Processing_materials,enrichment,10646,Enrichment,"[(33, 42)]",Work Advances at Iranian Uranium Enrichment Site,"[development, weaponization, reprocessing, enr...","[development, weaponization, reprocessing, enr...","[weaponization, reprocessing, development, NO,...",4,3
2,196,Locale_by_use,site,11550,Site,"[(44, 47)]",Work Advances at Iranian Uranium Enrichment Site,"[hedge, cemetery, village, countryside, center...","[hedge, cemetery, village, countryside, center...","[farm, campus, cemetery, theater, museum, work...",58,50
3,189,Quantified_mass,thousands,3328,thousands,"[(43, 51)]",Iran has begun preliminary work to install tho...,"[mountain, pinch, handful, ounce, mite, pile, ...","[mountain, pinch, handful, ounce, mite, pile, ...","[few, dose, stream, mountain, modicum, million...",44,44
4,620,Processing_materials,enrichment,10646,enrichment,"[(83, 92)]",Iran has begun preliminary work to install tho...,"[development, weaponization, reprocessing, enr...","[development, weaponization, reprocessing, enr...","[development, weaponization, reprocessing, NO,...",4,3
...,...,...,...,...,...,...,...,...,...,...,...,...
72412,203,Roadways,boulevard,3797,Boulevard,"[(32, 40)]","THE BARD was striding along the Boulevard , cl...","[line, highway, bridge, artery, underpass, roa...","[line, highway, bridge, artery, underpass, roa...","[course, parkway, rail, freeway, pavement, rai...",38,37
72413,203,Roadways,boulevard,3797,boulevard,"[(12, 20)]","In the main boulevard of Havana , they have ar...","[line, highway, bridge, artery, underpass, roa...","[line, highway, bridge, artery, underpass, roa...","[street, trail, flyover, bridge, pavement, und...",38,37
72414,203,Roadways,boulevard,3797,boulevard,"[(56, 64)]",He left the hospital and began to walk slowly ...,"[line, highway, bridge, artery, underpass, roa...","[line, highway, bridge, artery, underpass, roa...","[artery, motorway, cul-de-sac, railway, railro...",38,37
72415,203,Roadways,boulevard,3797,boulevards,"[(16, 25)]","Not for him the boulevards of Hollywood , nor ...","[line, highway, bridge, artery, underpass, roa...","[line, highway, bridge, artery, underpass, roa...","[underpass, artery, parkway, railroad, bridge,...",38,37


In [166]:
 dfg['final_preds'].iloc[3]

['few',
 'dose',
 'stream',
 'mountain',
 'modicum',
 'millions',
 'thousands',
 'plethora',
 'abundance',
 'torrent',
 'smattering',
 'wave',
 'number',
 'shitload',
 'amount',
 'touch',
 'pile',
 'trickle',
 'degree',
 'hundreds',
 'heap',
 'measure',
 'deal',
 'load',
 'quantity',
 'scads',
 'myriad',
 'mass',
 'many',
 'trace',
 'handful',
 'raft',
 'scores',
 'flood',
 'oodles',
 'ton',
 'deluge',
 'mite',
 'pinch',
 'several',
 'ounce',
 'dozens',
 'avalanche',
 'billions',
 'NO',
 'NO',
 'NO',
 'NO',
 'NO',
 'NO']

In [167]:
lemmatize(['thousands'])

['thousand']